In [2]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium

In [3]:
#get mrt data
mrt_df = pd.read_csv('../../data/mrt_station_final.csv', index_col=0)
mrt_df.head()
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()

# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
# response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()


Write a function for fetching isochrome. Configure params here

In [4]:
def fetch_isochrone(lat, lon, modes, cutoff="20M01S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-06-01T18:00:00+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None

x = fetch_isochrone(1.293657725, 103.8550812, modes = "TRANSIT")
features = x['features'][0]['geometry']['coordinates'][0][0]
print(features)

[[103.861, 1.286332], [103.861622, 1.286649], [103.862799, 1.287064], [103.863292, 1.287825], [103.864598, 1.289076], [103.865409, 1.289624], [103.8653, 1.290326], [103.865128, 1.291423], [103.865524, 1.292348], [103.865741, 1.293221], [103.865497, 1.29412], [103.864598, 1.293448], [103.863638, 1.29406], [103.862799, 1.294406], [103.862571, 1.294793], [103.862529, 1.29502], [103.862281, 1.296301], [103.862068, 1.296819], [103.862264, 1.298082], [103.862268, 1.298617], [103.862559, 1.300176], [103.86242, 1.300416], [103.862235, 1.301651], [103.861, 1.302118], [103.860943, 1.302158], [103.8592, 1.302174], [103.859171, 1.302185], [103.859053, 1.302214], [103.857872, 1.302685], [103.857401, 1.30274], [103.85616, 1.302772], [103.855602, 1.302732], [103.853983, 1.302394], [103.853803, 1.302402], [103.85346, 1.302214], [103.852004, 1.301635], [103.851056, 1.301266], [103.850205, 1.301106], [103.84884, 1.30085], [103.848406, 1.302116], [103.847844, 1.300416], [103.846607, 1.299325], [103.84610

DATA NEEDED:

Isochrones: 10min-60min, 1min interval (ideal) else 5min interval

1. BICYCLE
2. BUS+ WALK
3. RAIL+ WALK
4. TRANSIT + WALK
5. CAR

Set up function for saving all isochrones by transport mode

In [7]:
def fetch_all_isochrones(df, interval):
    """Fetch and append bicycle isochrones for each MRT station and specified time intervals."""
    # Define the cutoff intervals (in seconds) - lets us choose 1, 5, or 10 minute intervals later on
    cutoff_intervals = [f"{x}M" for x in range(10, 61, interval)]
    
    # Iterate over the cutoff intervals
    for interval in cutoff_intervals:
        # Define a column name for the interval
        column_name = f"isochrone_{interval}"
        # Initialize the column with empty lists
        df[column_name] = None
        
        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            # Fetch the isochrone GeoJSON for the current MRT station and cutoff interval
            isochrone_geojson = fetch_isochrone(row['Latitude'], row['Longitude'], modes="BICYCLE", cutoff=interval )
            
            # Check if the response is not None
            if isochrone_geojson:
                # Extract the coordinates of the first feature (assuming single isochrone feature per request)
                if 'features' in isochrone_geojson and len(isochrone_geojson['features']) > 0:
                    features = isochrone_geojson['features'][0]['geometry']['coordinates'][0][0]
                    # Assign the extracted features to the corresponding DataFrame cell
                    df.at[index, column_name] = features
                print(f"Fetched isochrone for interval {interval}, MRT: {row['MRT.Name']}")
            else:
                print(f"Failed to fetch isochrone for interval {interval}, MRT: {row['MRT.Name']}")
    
    return df

# Note: Ensure that mrt_df already exists and contains 'MRT.Name', 'Latitude', and 'Longitude' columns


In [8]:
# Apply the function to the mrt_df DataFrame
bicycle_isochrones = mrt_df.copy()
bicycle_isochrones = fetch_all_isochrones(bicycle_isochrones, interval=5)
bicycle_isochrones.to_json('../../data/Isochrone_data/bicycle_isochrones.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [5]:

m = folium.Map(location=coords[0], zoom_start=12)

# Fetch and add isochrones for each MRT station
for lat, lon in coords:
    isochrone_geojson = fetch_isochrone(lat, lon)
    if isochrone_geojson:
        folium.GeoJson(isochrone_geojson, name=f"Isochrone {lat}, {lon}").add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m


TypeError: fetch_isochrone() missing 1 required positional argument: 'modes'